# Создание простой строковой базы данных в Python

Учебный блокнот: как сделать простую «базу данных» на строках и файлах без настоящей СУБД.

## План занятия
1. Идея строковой базы данных
2. Представление записей в памяти (список строк, список словарей)
3. Базовые операции: добавление, просмотр, поиск, изменение, удаление (CRUD)
4. Сохранение в текстовый файл (формат CSV / «одна строка — одна запись»)
5. Загрузка из файла
6. Поиск по подстроке и с помощью регулярных выражений
7. Мини‑проект: мини‑справочник (телефонная книга / база студентов)

Во всех примерах используем **Python 3**.

## 1. Идея строковой базы данных
Настоящие базы данных (SQLite, PostgreSQL и др.) — это отдельные системы.
Но для простых учебных задач можно хранить данные:

* в **оперативной памяти** (списки, словари)
* в **текстовых файлах** (одна строка — одна запись)

Пример задачи: хотим хранить список студентов:

- Фамилия
- Имя
- Группа
- Средний балл

Каждую запись можно хранить либо как **строку**, либо как **словарь**.

## 2. Хранение записей в памяти
### Вариант 1. Список строк
Каждая строка содержит данные в виде:
`Фамилия;Имя;Группа;Средний_балл`

Пример:
```text
Иванов;Иван;ИВТ-101;4.5
Петров;Петр;ИВТ-102;3.8
```

### Вариант 2. Список словарей
```python
student = {
    'last_name': 'Иванов',
    'first_name': 'Иван',
    'group': 'ИВТ-101',
    'gpa': 4.5
}
```

В этом блокноте мы будем **хранить в памяти как словари**, а в **файле — как строки**.

In [ ]:
# Пример структуры данных в памяти
students = [
    {'last_name': 'Иванов', 'first_name': 'Иван', 'group': 'ИВТ-101', 'gpa': 4.5},
    {'last_name': 'Петров', 'first_name': 'Петр', 'group': 'ИВТ-102', 'gpa': 3.8},
]

for s in students:
    print(f"{s['last_name']} {s['first_name']} ({s['group']}), ср. балл: {s['gpa']}")

## 3. Базовые операции (CRUD)
CRUD — это аббревиатура:

- **C**reate — создание записи (добавление)
- **R**ead — чтение (просмотр) записей
- **U**pdate — обновление (изменение) записи
- **D**elete — удаление записи

Сделаем набор функций для работы с нашей «базой» в памяти.

In [2]:
# База данных в памяти: список словарей
db = []  # глобальная переменная для простоты примера

def create_student(last_name: str, first_name: str, group: str, gpa: float):
    """Добавить студента в базу."""
    student = {
        'last_name': last_name,
        'first_name': first_name,
        'group': group,
        'gpa': float(gpa),
    }
    db.append(student)

def read_all():
    """Вернуть список всех студентов."""
    return db

def update_student(index: int, **fields):
    """Обновить поля записи по индексу (если он существует)."""
    if 0 <= index < len(db):
        db[index].update(fields)
        return True
    return False

def delete_student(index: int) -> bool:
    """Удалить запись по индексу."""
    if 0 <= index < len(db):
        db.pop(index)
        return True
    return False

# Тестируем функции
create_student('Иванов', 'Иван', 'ИВТ-101', 4.5)
create_student('Петров', 'Петр', 'ИВТ-102', 3.8)
create_student('Сидорова', 'Анна', 'ИВТ-101', 4.2)

print('Все записи:')
for i, s in enumerate(read_all()):
    print(i, s)

print('\nОбновим средний балл записи 1:')
update_student(1, gpa=4.0)
print(db[1])

print('\nУдалим запись с индексом 0:')
delete_student(0)
for i, s in enumerate(read_all()):
    print(i, s)

Все записи:
0 {'last_name': 'Иванов', 'first_name': 'Иван', 'group': 'ИВТ-101', 'gpa': 4.5}
1 {'last_name': 'Петров', 'first_name': 'Петр', 'group': 'ИВТ-102', 'gpa': 3.8}
2 {'last_name': 'Сидорова', 'first_name': 'Анна', 'group': 'ИВТ-101', 'gpa': 4.2}

Обновим средний балл записи 1:
{'last_name': 'Петров', 'first_name': 'Петр', 'group': 'ИВТ-102', 'gpa': 4.0}

Удалим запись с индексом 0:
0 {'last_name': 'Петров', 'first_name': 'Петр', 'group': 'ИВТ-102', 'gpa': 4.0}
1 {'last_name': 'Сидорова', 'first_name': 'Анна', 'group': 'ИВТ-101', 'gpa': 4.2}


## 4. Сохранение строковой базы данных в файл
Для хранения на диске используем простой текстовый формат:

`Фамилия;Имя;Группа;Средний_балл`

Каждому студенту соответствует **одна строка**.
Файл пусть называется, например, `students_db.txt`.

In [4]:
import os

DB_FILENAME = 'students_db.txt'

def student_to_line(student: dict) -> str:
    """Преобразовать словарь в строку для записи в файл."""
    return f"{student['last_name']};{student['first_name']};{student['group']};{student['gpa']}"

def line_to_student(line: str) -> dict:
    """Преобразовать строку из файла в словарь."""
    parts = line.strip().split(';')
    if len(parts) != 4:
        raise ValueError('Неверный формат строки: ' + line)
    last_name, first_name, group, gpa = parts
    return {
        'last_name': last_name,
        'first_name': first_name,
        'group': group,
        'gpa': float(gpa),
    }

def save_db_to_file(filename: str = DB_FILENAME):
    with open(filename, 'w', encoding='utf-8') as f:
        for student in db:
            f.write(student_to_line(student) + '\n')
    print(f'База данных сохранена в файл {filename}')

def load_db_from_file(filename: str = DB_FILENAME):
    global db
    db = []
    if not os.path.exists(filename):
        print('Файл не найден, начинаем с пустой базы')
        return
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                db.append(line_to_student(line))
    print(f'Загружено записей: {len(db)} из файла {filename}')

# Сохраним текущую базу в файл и перечитаем её
save_db_to_file()
db = []  # очищаем
print('После очистки db =', db)
load_db_from_file()
print('После загрузки:')
for s in db:
    print(s)

База данных сохранена в файл students_db.txt
После очистки db = []
Загружено записей: 2 из файла students_db.txt
После загрузки:
{'last_name': 'Петров', 'first_name': 'Петр', 'group': 'ИВТ-102', 'gpa': 4.0}
{'last_name': 'Сидорова', 'first_name': 'Анна', 'group': 'ИВТ-101', 'gpa': 4.2}


## 5. Поиск по строковой базе данных
Сделаем функции поиска:

1. Поиск по **подстроке** в фамилии или имени
2. Поиск по **регулярному выражению** (например, по группе или другим полям)

Это уже приближается к идее «запросов» к базе данных.

In [ ]:
import re

def find_by_name_substring(sub: str):
    """Найти записи, где подстрока есть в фамилии или имени (без учета регистра)."""
    result = []
    sub_lower = sub.lower()
    for student in db:
        if sub_lower in student['last_name'].lower() or sub_lower in student['first_name'].lower():
            result.append(student)
    return result

def find_by_group_regex(pattern: str):
    """Найти записи, где группа удовлетворяет шаблону регулярного выражения."""
    result = []
    regex = re.compile(pattern)
    for student in db:
        if regex.search(student['group']):
            result.append(student)
    return result

# Примеры использования
print('Поиск по подстроке "ан":')
for s in find_by_name_substring('ан'):
    print(s)

print('\nПоиск по regex группы "^ИВТ-10[12]$":')
for s in find_by_group_regex(r'^ИВТ-10[12]$'):
    print(s)

## 6. Мини‑проект: консольное меню для работы с базой
Сделаем простой текстовый интерфейс:

1. Показать все записи
2. Добавить запись
3. Найти по подстроке в имени/фамилии
4. Сохранить в файл
5. Загрузить из файла
0. Выход

Такой подход можно использовать как основу для более серьёзных проектов.

In [9]:
def print_menu():
    print('\nМЕНЮ:')
    print('1 — Показать все записи')
    print('2 — Добавить запись')
    print('3 — Поиск по подстроке (имя/фамилия)')
    print('4 — Сохранить в файл')
    print('5 — Загрузить из файла')
    print('0 — Выход')

def print_all_students():
    if not db:
        print('База пуста')
        return
    for i, s in enumerate(db):
        print(f"{i}: {s['last_name']} {s['first_name']} ({s['group']}), ср. балл {s['gpa']}")

def interactive_loop():
    """Простое консольное меню. Работает в классическом интерпретаторе / терминале."""
    while True:
        print_menu()
        choice = input('Выберите пункт меню: ')
        if choice == '1':
            print_all_students()
        elif choice == '2':
            ln = input('Фамилия: ')
            fn = input('Имя: ')
            gr = input('Группа: ')
            gpa = float(input('Средний балл: '))
            create_student(ln, fn, gr, gpa)
            print('Запись добавлена')
        elif choice == '3':
            sub = input('Введите подстроку для поиска: ')
            results = find_by_name_substring(sub)
            if not results:
                print('Ничего не найдено')
            else:
                for s in results:
                    print(f"{s['last_name']} {s['first_name']} ({s['group']}), ср. балл {s['gpa']}")
        elif choice == '4':
            save_db_to_file()
        elif choice == '5':
            load_db_from_file()
        elif choice == '0':
            print('Выход из программы')
            break
        else:
            print('Неизвестный пункт меню')

print('Функция interactive_loop() создана.\n'
      'Для запуска консольного меню в обычном терминале вызовите interactive_loop().')

Функция interactive_loop() создана.
Для запуска консольного меню в обычном терминале вызовите interactive_loop().


In [ ]:
# Обновленная функция создания студента
def create_student(last_name: str, first_name: str, group: str, gpa: float, enrollment_year: int, record_book_number: str):
    """Добавить студента с расширенными полями."""
    student = {
        'last_name': last_name,
        'first_name': first_name,
        'group': group,
        'gpa': float(gpa),
        'enrollment_year': enrollment_year,
        'record_book_number': record_book_number,
    }
    db.append(student)

# Обновленная функция преобразования в строку для файла
def student_to_line(student: dict) -> str:
    return f"{student['last_name']};{student['first_name']};{student['group']};{student['gpa']};{student['enrollment_year']};{student['record_book_number']}"

# Обновленная функция загрузки из строки
def line_to_student(line: str) -> dict:
    parts = line.strip().split(';')
    if len(parts) != 6:
        raise ValueError('Неверный формат строки: ' + line)
    last_name, first_name, group, gpa, enrollment_year, record_book_number = parts
    return {
        'last_name': last_name,
        'first_name': first_name,
        'group': group,
        'gpa': float(gpa),
        'enrollment_year': int(enrollment_year),
        'record_book_number': record_book_number,
    }


In [15]:
def find_by_gpa(min_gpa: float):
    return [s for s in db if s['gpa'] > min_gpa]

results = find_by_gpa(4.0)
print(s)

{'last_name': 'Сидорова', 'first_name': 'Анна', 'group': 'ИВТ-101', 'gpa': 4.2}


In [ ]:
def delete_by_group(group: str):
    global db
    db = [s for s in db if s['group'] != group]
    print(f'Удалены все студенты группы {group}.')

delete_by_group('ИВТ-101')

Удалены все студенты группы ИВТ-101.


In [17]:
import csv

def export_to_csv(filename: str):
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=['last_name', 'first_name', 'group', 'gpa', 'year', 'record_book'])
        writer.writeheader()
        writer.writerows(db)

export_to_csv('students.csv')
print("База данных экспортирована в students.csv")


База данных экспортирована в students.csv


In [ ]:
def find_by_regex(field: str, pattern: str):
    import re
    regex = re.compile(pattern)
    result = []
    for s in db:
        if field in s and regex.search(str(s[field])):
            result.append(s)
    return result

# поиск по группе, начинающейся на "ИВТ-10":
find_by_regex('group', r'^ИВТ-10')

[{'last_name': 'Петров', 'first_name': 'Петр', 'group': 'ИВТ-102', 'gpa': 4.0}]

## 7. Практические задания
Рекомендуется выполнить прямо в этом блокноте, добавляя новые ячейки.

### Задание 1. Расширение структуры данных
Добавьте к записи студента поля:
- год поступления
- номер зачётной книжки
Настройте функции `create_student`, `student_to_line`, `line_to_student`, чтобы эти поля тоже сохранялись.

### Задание 2. Поиск по среднему баллу
Сделайте функцию, которая находит всех студентов с средним баллом **выше заданного**.
Например: `find_by_gpa(min_gpa: float)`.

### Задание 3. Удаление по условию
Реализуйте функцию, которая удаляет всех студентов заданной группы.
Например: `delete_by_group(group: str)`.

### Задание 4. Экспорт в CSV
Сохраните базу данных в файл формата CSV (через модуль `csv`).
Сравните этот подход с ручной записью строк.

### Задание 5. Поиск по регулярному выражению
Сделайте более универсальную функцию поиска `find_by_regex(field, pattern)`,
которая принимает имя поля (например, `'last_name'` или `'group'`) и шаблон регулярного выражения.
Используйте её для разных поисковых запросов.